# EEEN30101 Numerical Analysis

# Week 02

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides more information about Heron's algorithm, and introduces three of the most established root finding algorithms (i.e., bisection algorithm, secant method, and Newton-Raphson's method). 

That is, the topics covered in this notebook include:
- Heron's method - Convergence
- Bisection algorithm
- Secant method
- Newton-Raphson's method

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as you can explore different variations of the examples, which will better prepare you for the exams.

## List of contents

- [Heron's method - Convergence](#Heron's-method---Convergence)
  - [Approximating the equation](#Approximating-the-equation)
  - [Analysing convergence properties](#Analysing-convergence-properties)
  - [Testing Heron's equation](#Testing-Heron's-equation)


- [Root finding algorithms and the bisection algorithm](#Root-finding-algorithms-and-the-bisection-algorithm)
  - [Solving a familiar problem](#Solving-a-familiar-problem)
  - [Analysing the locations of roots](#Analysing-the-locations-of-roots)
  - [Bisection algorithm](#Bisection-algorithm)


- [Secant method](#Secant-method)
  - [Formulation of the Secant method](#Formulation-of-the-Secant-method)
  - [Testing the Secant method](#Testing-the-Secant-method)


- [Newton-Raphson's method](#Newton-Raphson's-method)
  - [Formulation of Newton's method](#Formulation-of-Newton's-method)
  - [Testing Newton's method](#Testing-Newton's-method)
  - [Special applications of Newton's method](#Special-applications-of-Newton's-method)
  - [Computational efficiency of Newton's method](#Computational-efficiency-of-Newton's-method)


- [Conclusion](#Conclusion)

## Before we begin

Before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30101 Week 2 
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

This notebook provides some examples in python, for that purpose the following libraries will be loaded:

In [1]:
import numpy  # To define and use matrices
import matplotlib.pyplot as plt  # To plot figures
import math

try:
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    import ipywidgets as widgets
from ipywidgets import interact

[Back to top](#EEEN30101-Numerical-Analysis)

## Heron's method - Convergence

As was presented last week, according to Heron's, the square root of a number ($S$) can be iteratively estimated by first making a guess about the value of the square root ($x_k$) and, afterwards, improving our guess ($x_{k+1}$) by solving the following equation iteratively.

$$x_{k+1} = \frac{x_k+S/x_k}{2}$$

We also established that, if Heron's algorithm converges, the result should converge to $\sqrt{S}$

This time, we will now explore if the algorithm actually converges. That is, if the difference between successive guesses ($|x_{k+1} - x_k|$) is gradually becoming smaller.

### Approximating the equation

It is not straightforward to calculate $|x_{k+1} - x_k|$ from Heron's method, so it is convenient to express Heron's formula in a different way. For example, we can express it as a series of polynomials based on Taylor's expansion.

First, let's redefine Heron's equation as a typical function $g(x)$:
$$g(x) = \frac{x+S/x}{2}$$

And let us expand this equation using Taylor series around a selected point ($\xi$):

$$
\begin{aligned}
  g(x) & = \frac{x+S/x}{2} \\
   & = \sum_{n=0}^{n=\infty}{\frac{g^n(\xi)}{n!} (x-\xi)^n} \\ 
   & = g(\xi) + g'(\xi)(x-\xi) + \frac{g''(\xi)}{2!}(x-\xi)^2 + ... + \frac{g^n(\xi)}{n!} (x-\xi)^n
\end{aligned}
$$


One of the key advantages of Taylor's expansion is that, according to Taylor's theorem, the value of the higher order polynomials tends to zero. In other words, it is generally reasonable to just use the first two elements of the equation:

$$g(x) = \frac{x+S/x}{2} \approx g(\xi) + g'(\xi)(x-\xi)$$

><mark>This approximation is easier to use than the full Heron's equation</mark>

To illustrate what we just did, let us compare Heron's function with different Taylor expansion approximations, specifically:
- Herons equation:
$$Heron(x) = \frac{x+S/x}{2}$$</p>


- A Taylor expansion of Heron's equation with the first two elements:

$$Taylor2(x) = \frac{\xi+S/\xi}{2} + \frac{1-S/\xi^2}{2}(x-\xi)$$


- A Taylor expansion of Heron's equation with the first three elements:

$$Taylor3(x) = \frac{\xi+S/\xi}{2} + \frac{1-S/\xi^2}{2}(x-\xi) + \frac{S}{\xi^32!}(x-\xi)^2 $$


- A Taylor expansion of Heron's equation with the first four elements:

$$Taylor4(x) = \frac{\xi+S/\xi}{2} + \frac{1-S/\xi^2}{2}(x-\xi) + \frac{S}{\xi^32!}(x-\xi)^2 + \frac{-3S}{\xi^43!}(x-\xi)^3$$


- A Taylor expansion of Heron's equation with the first five elements:

$$Taylor5(x) = \frac{\xi+S/\xi}{2} + \frac{1-S/\xi^2}{2}(x-\xi) + \frac{S}{\xi^32!}(x-\xi)^2 + \frac{-3S}{\xi^43!}(x-\xi)^3 + \frac{12S}{\xi^54!}(x-\xi)^4$$

[Back to top](#EEEN30101-Numerical-Analysis)

In [2]:
@interact
def TestTaylor(S = widgets.FloatSlider(min=5, max=15,step=0.1,value=10, description='S', continuous_update=False),
               a = widgets.FloatSlider(min=5, max=15,step=0.1,value=10, description='𝜉', continuous_update=False)):
    T = 100
    X = numpy.linspace(1,20,T)

    Heron = numpy.zeros(T)
    Taylor2 = numpy.zeros(T)
    Taylor3 = numpy.zeros(T)
    Taylor4 = numpy.zeros(T)
    Taylor5 = numpy.zeros(T)

    for n in range(T):
        x = X[n]
        Heron[n] = g = (x+S/x)/2
        Taylor2[n] = (a+S/a)/2 + (1-S/a**2)/2*(x-a) 
        Taylor3[n] = Taylor2[n] + S/a**3*(x-a)**2/math.factorial(2) 
        Taylor4[n] = Taylor3[n] + -3*S/a**4*(x-a)**3/math.factorial(3) 
        Taylor5[n] = Taylor4[n] + 12*S/a**5*(x-a)**4/math.factorial(4)

    fig, ax = plt.subplots()
    ax.plot(X, Heron, '--', label='Heron')
    ax.plot(X, Taylor2, label='Taylor2')
    ax.plot(X, Taylor3, label='Taylor3')
    ax.plot(X, Taylor4, label='Taylor4')
    ax.plot(X, Taylor5, label='Taylor5')

    ax.set(xlabel='Guess', ylabel='Updated guess')
    ax.grid()
    plt.legend()
    plt.show()


interactive(children=(FloatSlider(value=10.0, continuous_update=False, description='S', max=15.0, min=5.0), Fl…

From the simulations, it can be concluded that:
- The use of Taylor series to approximate Heron's formula is reasonable.
- Accuracy is affected by the point selected to perform the series expansion ($a$).

[Back to top](#EEEN30101-Numerical-Analysis)

### Analysing convergence properties

Now that Heron's formula has been approximated as a series of polynomials, we can estimate $x_k$ and $x_{k+1}$ as:

$$x_k \approx g(\xi) + g'(x_{k-1}-\xi)$$

$$x_{k+1} \approx g(\xi) + g'(x_k-\xi)$$

Therefore, $|x_{k+1} - x_k|$ can be calculated as:

$$
\begin{aligned}
  |x_{k+1} - x_k| & \approx |g(\xi) + g'(x_k-\xi) - g(\xi) - g'(x_{k-1}-\xi)| \\
   & \approx | g'(x_k-\xi) - g'(x_{k-1}-\xi)| \\ 
   & \approx |x_k - x_{k-1}||g'(\xi)|
\end{aligned}
$$


Based on the last equation, for the difference between consecutive guesses ($|x_k - x_{k-1}|$) to gradually decrease, $|g'(\xi)|$ must be lower than one for all possible $\xi$.

So let us examine, $g'(x)$, which is the differential of Heron's formula:

$$\frac{\partial Heron(x)}{\partial x} =\frac{\partial}{\partial z} \left[ \frac{x+S/x}{2} \right] = \frac{1}{2} \left(1 - \frac{S}{x^2} \right)$$


><mark>Note that convergence speed will increase if the magnitude of this equation is low, e.g., when $x \approx S$  </mark>

The absolute value of this equation must be lower than one. Therefore:
$$-1 < \frac{1}{2} \left(1 - \frac{S}{x^2} \right) < 1$$

This is the same as:
$$-3 < - \frac{S}{x^2} < 1$$

The right side of the equation can be rewritten as:
$$-S < x^2$$

><mark>This condition will be true as long as $S>0$</mark>

The left side of the equation can be rewritten as:
$$3 > \frac{S}{x^2}$$

$$x^2 > \frac{S}{3}$$

For this to be true **S has to be greater than zero** and the value of $x$ has to be derived from Heron's formula as follows:

$$
\begin{aligned}
  x_{k+1}^2 & = \frac{(x_k+S/x_k)^2}{4} \\
   & = \frac{1}{4}\left(x_k^2+2S+\frac{S^2}{x_k^2} \right) \\ 
   & = \frac{S}{2} + \frac{1}{4}\left(x_k^2+\frac{S^2}{x_k^2} \right)
\end{aligned}
$$


From this last equation, we know that $x_{k+1}^2$ will be equal to $S/2$ plus some additional values. In other words, considering that the additional values can be zero or more, we can state that x will be greater than $S/2$ and, thus, also greater than $S/3$:

$$x_{k+1}^2 \geq \frac{S}{3}$$

[Back to top](#EEEN30101-Numerical-Analysis)

### Testing Heron's equation

Based on the information provided above, it can be concluded that Heron's algorithm:
- should converge as long as $S \geq 0$
- should converge faster when $x \approx S$

You can test the algorithm using the tool below.

In [3]:
def Heron(Number, Guess, Threshold, flg=True):
    Error = 1000
    if flg:
        print('     Input  Estimated     Error:')
        print('   number:      root:')
        print('%10.4f %10.4f'%(Number, Guess))

    iterations = 0
    while Error > Threshold:
        iterations+=1
        NewGuess = (Guess+Number/Guess)/2
        Error = abs(NewGuess-Guess)
        if flg:
            print('%10.4f %10.4f %10.6f'%(Number, NewGuess, Error))
        Guess = NewGuess
        
        if iterations >=100:
            Error = 0
    if flg:
        if iterations <100:
            print('Converged after %d iterations'%iterations)
        else:
            print('Failed to converge after %d iterations'%iterations)

    return NewGuess, iterations, Error

@interact
def Heron_Interactive(Number=widgets.FloatText(value=10, description='Input'),
          Guess=widgets.FloatText(value=2, description='Guess'),
          Threshold=widgets.FloatText(min=0.000001, value=0.001, description='Threshold')):
    NewGuess, iteration, Error = Heron(Number, Guess, Threshold)

interactive(children=(FloatText(value=10.0, description='Input'), FloatText(value=2.0, description='Guess'), F…

[Back to top](#EEEN30101-Numerical-Analysis)

## Root finding algorithms and the bisection algorithm

The Bisection algorithm is one of the key foundations of root finding approaches, which, as their name suggest are approaches developed to find the roots of a given equation.

Let us use an example to illustrate how root finding approaches work, and introduce the bisection method.

### Solving a familiar problem

As we have been analysing square roots with Heron's algorithm, let us use a square root equation as an example:

$$x = \sqrt{S}$$

We should first rewrite the equation so that it is equal to zero as done below. 

$$x^2 - S = 0$$

The roots of the algorithm are the values of the unknown variables ($x$ in this case) that satisfy the equation.


We need to assume some numbers for our example. For instance, assume $S=2$:

In [4]:
S = 2

print('S is set to: %.4f'%S)

S is set to: 2.0000


As this is a quadratic equation, and its solution is also the square root of ($S$), we should already know a couple of options to solve this problem, such as:

- The general quadratic formula:
$$ax^2+bx+c=0$$

$$x=\frac{-b \pm \sqrt{b^2-4ac}}{2a}$$

In [5]:
@interact
def Heron_Interactive(a=widgets.FloatText(value=1, description='a'),
          b=widgets.FloatText(value=0, description='b'),
          c=widgets.FloatText(value=-S, description='c')):

    x = (-b + numpy.sqrt(b**2-4*a*c))/2/a
    print('The square root is: %.4f'%x)

interactive(children=(FloatText(value=1.0, description='a'), FloatText(value=0.0, description='b'), FloatText(…

- Heron's algorithm
$$x_{k+1} = \frac{x_k+S/x_k}{2}$$

In [6]:
@interact
def Heron_Interactive(Number=widgets.FloatText(value=S, description='Input'),
          Guess=widgets.FloatText(value=2, description='Guess'),
          Threshold=widgets.FloatText(min=0.000001, value=0.001, description='Threshold')):

    NewGuess, iteration, Error = Heron(Number, Guess, Threshold, False)
    print('The square root is: %.4f'%NewGuess)

interactive(children=(FloatText(value=2.0, description='Input'), FloatText(value=2.0, description='Guess'), Fl…

However, would you be able to get the roots of other equations? Some of these other equations may be more complex, or you may not have defined yet.

We need a more general approach to obtain roots in such cases.

[Back to top](#EEEN30101-Numerical-Analysis)

### Analysing the locations of roots

Assume now that we do not know a specific method to obtain the roots of our equation, how would you solve the problem?

Let's continue with our example $x^2 - S = 0$ but, this time, we will analyse the function.

For convenience, we will use a python method that will handle the equations for us. This will also allow us to seamlessly customise our examples later on.

> Do not worry about the code, it is here for completeness, but you do not need to understand it and it will not be marked.

In [7]:
class polynomial:
    def __init__(self, arg):
        '''Save parameters of polynomial'''
        self.pars = arg
        self.Nopars = len(arg)

    def set_x(self, x):
        '''Set x values for polynomial'''
        self.x = x
        self.Nox = len(x)

    def get_y(self):
        '''Get the values of f(x)'''
        self.y = numpy.zeros((self.Nox))
        for n in range(self.Nox):
            for p in range(self.Nopars):
                self.y[n] += self.pars[p]*self.x[n]**(self.Nopars-p-1)
        return self.y
    
    def get_diff(self):
        '''Get the values of f'(x)'''
        self.dy = numpy.zeros((self.Nox))
        for n in range(self.Nox):
            for p in range(self.Nopars-1):
                self.dy[n] += (self.Nopars-p-1)*self.pars[p]*self.x[n]**(self.Nopars-p-2)
        return self.dy

    def display(self):
        for p in range(self.Nopars):
            if self.pars[p] != 0:
                if p == 0:
                    print('%d'%self.pars[p], end='')
                else:
                    if self.pars[p]>=0:
                        print(' + ',self.pars[p] ,end='')
                    else:
                        print(' - ',-self.pars[p] ,end='')
                aux = self.Nopars-p-1
                if aux>1:
                    print('x^%d'%aux, end='')
                elif aux==1:
                    print('x', end='')

Example = polynomial([1, 0, -S])
print('The polynomial considered in this example is:\nf(x) = ', end='')
Example.display()

The polynomial considered in this example is:
f(x) = 1x^2 -  2

Check the figure of our example presented below.

In [8]:
@interact
def PlotFigure(From = widgets.FloatSlider(min=-10, max=0, value=0, description='From', continuous_update=False),
               To = widgets.FloatSlider(min=1, max=10, value=2, description='To', continuous_update=False)):
    T = 100
    X = numpy.linspace(From, To, T)

    Example.set_x(X)
    y = Example.get_y()

    fig, ax = plt.subplots()
    ax.plot(X,y, '--', label='Polynomial')
    ax.plot([From, To],[0, 0], label='x-axis')

    ax.set(xlabel='(x-axis)', ylabel='(y-axis)')
    ax.grid()
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='From', max=0.0, min=-10.0),…

After analysing the figure, you can observe that the root of the equation is the value of $x$ that makes the function intersect the x axis.

Based on the above information, there are different approaches that can be used to find roots. Let us begin with the Bisection algorithm.

[Back to top](#EEEN30101-Numerical-Analysis)

### Bisection algorithm

Based on our observations, if we know that the roots are located in places where the function intersects the x-axis, we can conclude that a root can be found between two $x$ values (i.e., $\underline{x}$ and $\bar{x}$) if:

- The values are different:

$$\underline{x} \leq \bar{x}$$

- One of their f(x) values is positive, whereas the other is negative:

$$f(\underline{x})f(\bar{x})<0$$


As we know that a root is located between $\underline{x}$ and $\bar{x}$, based on the Bisection algorithm, we can select the value in the middle of those two points as our new guess of the root ($\hat{x}$):

$$\hat{x} = \frac{1}{2}(\underline{x}+\bar{x})$$

This new estimate ($\hat{x}$) will be used to replace either $\underline{x}$ or $\bar{x}$ using the following rules:
- Do $f(\hat{x})$ and $f(\underline{x})$ have the same sign?
  - If so, $f(\hat{x})$ will replace $f(\underline{x})$, whereas $f(\bar{x})$ will remain the same for the next iteration.
  - If not ($f(\hat{x})$ and $f(\hat{x})$ have the same sign), $f(\hat{x})$ will replace $f(\bar{x})$, whereas $f(\underline{x})$ will remain the same for the next iteration

This can be expressed as follows:

$ \text{if} f(\hat{x}_k)f(\underline{x}_k)>0:$
$$\underline{x}_{k+1} = \hat{x}_k$$
$$\bar{x}_{k+1} = \bar{x}_k$$
$ \text{else}:$
$$\underline{x}_{k+1} = \underline{x}_k$$
$$\bar{x}_{k+1} = \hat{x}_k$$

The process will be repeated until the expected error is within an estimated threshold ($\varepsilon$):
$$Error = \frac{1}{2}(\bar{x}_k - \underline{x}_k) \leq \varepsilon$$

It is time to use Bisection to find a root of an equation. Use the python tool below to solve the current example, or other examples of your choice.

In [9]:
@interact
def TestBisection(Xmin = widgets.FloatSlider(min=-10, max=0, value=0, description='$xmin$', continuous_update=False),
               Xmax = widgets.FloatSlider(min=1, max=10, value=2, description='xmax', continuous_update=False)):
    Threshold = 0.0001
    Error = 0.5*(Xmax-Xmin)
       
    Example.set_x([Xmax])
    fxmax = Example.get_y()
    
    Example.set_x([Xmin])
    fxmin = Example.get_y()
    
    iteration = 0
    print('  k     xmin     xmax        x    Error  f(xmin)  f(xmax)     f(x)')
    while Error > Threshold:
        x = 0.5*(Xmax+Xmin)
    
        Example.set_x([x])
        fx = Example.get_y()
        
        print('%3.0f %8.4f %8.4f %8.4f %8.4f %8.4f %8.4f %8.4f'%
              (iteration, Xmin, Xmax, x, Error, fxmin[0], fxmax[0], fx[0]))
    
        if fx*fxmax>0:
            Xmax = x
            fxmax = fx
        else:
            Xmin = x
            fxmin = fx
        
        Error = 0.5*(Xmax-Xmin)
        
        #print (Xmin, Xmax)
        
        if iteration > 100:
            Error = 0
        iteration += 1
    print('\nA root of f(x) = (', end='')
    Example.display()
    print(') is %.4f'%x)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='$xmin$', max=0.0, min=-10.0…

Based on the results of the simulations, you may observe the following:
- The algorithm requires a large number of iterations
- The error bounds halve at each iteration (linear convergence)

[Back to top](#EEEN30101-Numerical-Analysis)

## Secant method

### Formulation of the Secant method

One of the disadvantage of the Bisection method is the need to find an initial range of values ($f(\hat{x})$ and $f(\bar{x})$) that include a root.

The Secant method tries to address this issue by drawing a line between the selected range of values an extending this secant line until it intercepts the x-axis.

$$x_{k+1} = \underline{x}_k-(\bar{x}_k - \underline{x}_k)\frac{f(\underline{x}_k)}{f(\bar{x}_k)-f(\underline{x}_k)}$$

Use the python method below to test this approach to create Secants.

In [10]:
@interact
def PlotSecant(From = widgets.FloatSlider(min=-10, max=10, value=-1, description='From', continuous_update=False),
               To = widgets.FloatSlider(min=-10, max=10, value=3, description='To', continuous_update=False),
              Xmin = widgets.FloatSlider(min=-2, max=2, value=0, description='xmin', continuous_update=False),
              Xmax = widgets.FloatSlider(min=-2, max=2, value=1, description='xmax', continuous_update=False)):
    T = 100
    X = numpy.linspace(From, To, T)

    Example.set_x(X)
    y = Example.get_y()
    
    if Xmin > Xmax:
        aux = Xmin
        Xmin = Xmax
        Xmax = aux
    elif Xmin==Xmax:
        Xmax = Xmin+0.001
    
    Example.set_x([Xmin])
    fxmin = Example.get_y()
    
    Example.set_x([Xmax])
    fxmax = Example.get_y()

    fig, ax = plt.subplots()
    ax.plot(X,y, '--', label='Polynomial')
    ax.plot([From, To],[0, 0], label='x-axis')
    ax.plot([Xmin],[fxmin],'*', label='xmin')
    ax.plot([Xmax],[fxmax],'*', label='xmax')
    
    x = Xmin-(Xmax-Xmin)*fxmin/(fxmax-fxmin)
    ax.plot([Xmin, x[0]],[fxmin[0], 0], label='Secant')
    
    #if fxmin*fxmax<0:
        #ax.plot([Xmin, Xmax],[fxmin, fxmax], label='Secant')
    #else:
        #Secant = Xmin-(Xmax-Xmin)*fxmin/(fxmax-fxmin)
        #if Secant>Xmin:
            #ax.plot([Xmin, Secant],[fxmin, 0], label='Secant')

    ax.set(xlabel='(x-axis)', ylabel='(y-axis)')
    ax.grid()
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=-1.0, continuous_update=False, description='From', max=10.0, min=-10.0…

[Back to top](#EEEN30101-Numerical-Analysis)

### Testing the Secant method

The distance between the two guesses used for the calculation of the secant affect the accuracy of the method, which should also affect the computational costs.

To analyse this effect, use the python method below, where only one guess is required as an input ($\underline{x}$), whereas the second guess is estimated as:

<p style="text-align: center;">$$\bar{x}_k = \underline{x}_k + \Delta$$</p>

In [11]:
@interact
def TestSecant(x = widgets.FloatSlider(min=-10, max=10, value=1, description='x', continuous_update=False),
              D = widgets.FloatSlider(min=0.0001, max=5, value=0.1, description='Δ', continuous_update=False)):
    Threshold = 0.0001
    
    iteration = 0
    Error = 1000
    print('  k     xmin     xmax        x    Error     f(x)')
    while Error > Threshold:
        
        Xmin = x
        Xmax = Xmin+D
        
        Example.set_x([Xmax])
        fxmax = Example.get_y()[0]

        Example.set_x([Xmin])
        fxmin = Example.get_y()[0]

        x = Xmin-(Xmax-Xmin)*fxmin/(fxmax-fxmin)
    
        Error = abs(x-Xmin)
        
        print('%3.0f %8.4f %8.4f %8.4f %8.4f %8.4f'%
              (iteration, Xmin, Xmax, x, Error, fxmin))
        
        if iteration > 100:
            Error = 0
        iteration += 1
    print('\nA root of f(x) = (', end='')
    Example.display()
    print(') is %.4f'%x)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='x', max=10.0, min=-10.0), F…

The results of the simulation show that the accuracy of the Secant method tends to improve when the difference between both guesses ($\Delta$) is small.

><mark>When $\Delta$ tends to zero, Secant is equivalent to the gradient of the equation (i.e., differential)</mark>

[Back to top](#EEEN30101-Numerical-Analysis)

## Newton-Raphson's method

### Formulation of Newton's method

Based on the information presented in the previous section, the accuracy of the Secant method tends to improve when the pairs of guesses used for the algorithm ($\bar{x}_k$ and $\underline{x}_k$) are close together.

Let us explore the implications of having the guesses very close together. For that purpose, let us begin with the equation of the Secant method:

$$x_{k+1} = \underline{x}_k-(\bar{x}_k - \underline{x}_k)\frac{f(\underline{x}_k)}{f(\bar{x}_k)-f(\underline{x}_k)}$$

And let us rewrite it like this:

$$x_{k+1} = \underline{x}_k-\frac{f(\underline{x}_k)}{F(x_k)}$$

Where:

$$F(x_k) = \frac{f(\bar{x}_k)-f(\underline{x}_k)}{\bar{x}_k - \underline{x}_k}$$

Now, let us make $\bar{x}_k$ and $\underline{x}_k$ close together by replacing them with $x$ and $x+\Delta$:

$$
\begin{aligned}
  F(x_k) & = \frac{f(x+\Delta)-f(x)}{x+\Delta -x} \\
   & = \frac{f(x+\Delta)-f(x)}{\Delta}
\end{aligned}
$$

And make $\Delta$ as small as possible:
$$\lim_{\Delta\to0} F(x_k) = \frac{f(x+\Delta)-f(x)}{\Delta} = f'(x)$$

We can now modify the Secant method with the following equation, which is the Newton-Raphson's method (or just Newton's method for short):

$$x_{k+1} = x_k-\frac{f(x)}{f'(x)}$$


[Back to top](#EEEN30101-Numerical-Analysis)

### Testing Newton's method

Now that we have the equation for Newton's method, we can analyse how the gradient ($f'(x)'$) can be used to find improved estimations of roots. For this purpose, use the python method below.

In [12]:
@interact
def PlotGradient(From = widgets.FloatSlider(min=-10, max=10, value=-1, description='From', continuous_update=False),
                 To = widgets.FloatSlider(min=-10, max=10, value=3, description='To', continuous_update=False),
                 x = widgets.FloatSlider(min=0, max=3, value=2, description='x', continuous_update=False)):
    T = 100
    X = numpy.linspace(From, To, T)

    Example.set_x(X)
    y = Example.get_y()
    
    Example.set_x([x])
    fx = Example.get_y()
    dfx = Example.get_diff()
    

    fig, ax = plt.subplots()
    ax.plot(X,y, '--', label='Polynomial')
    ax.plot([From, To],[0, 0], label='x-axis')
    ax.plot([x],[fx],'*', label='x')
    if dfx == 0:
        ax.plot([From, To],[fx[0], fx[0]], label='Gradient')
    else:
        xk = x - fx/dfx
        ax.plot([x, xk[0]],[fx[0], 0], label='Gradient')

    ax.set(xlabel='(x-axis)', ylabel='(y-axis)')
    ax.grid()
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=-1.0, continuous_update=False, description='From', max=10.0, min=-10.0…

If the process is repeated iteratively (as done with the Bisection and Secant methods), we can estimate the root of an equation within a tolerable margin of error.

Use the python method below to test Newton's method:

In [13]:
@interact
def TestNewton(x = widgets.FloatSlider(min=-10, max=10, value=1, description='x', continuous_update=False)):
    Threshold = 0.0001
    
    iteration = 0
    Error = 1000
    print('  k        x    Error     f(x)')
    while Error > Threshold:
        
        Example.set_x([x])
        fx = Example.get_y()[0]
        dfx = Example.get_diff()[0]

        xnew = x-fx/dfx
        Error = abs(xnew-x)
        x = xnew
        
        print('%3.0f %8.4f %8.4f %8.4f'%
              (iteration, x, Error, fx))
        
        if iteration > 100:
            Error = 0
        iteration += 1
    print('\nA root of f(x) = (', end='')
    Example.display()
    print(') is %.4f'%x)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='x', max=10.0, min=-10.0), O…

From the simulations, it should be apparent than Newton's method seems to converge faster than the other root finding algorithms, and more or less at the same speed as Heron's method (only applicable to square roots). 

[Back to top](#EEEN30101-Numerical-Analysis)

### Special applications of Newton's method

As mentioned above, the convergence speeds of Newton's and Heron's method seem very similar. This is because Heron's method is a special application of Newton's method.

To illustrate this, let us begin with the quadratic equation that matches the same problem that Heron's method solves (i.e., the square root of a number $S$).
$$f(x) = x^2-S$$

Let us now get the differential of this equation:

$$f'(x) = 2x$$

And apply Newton's method:

$$
\begin{aligned}
  g(x) & = x-\frac{f(x)}{f'(x)} \\
   & = x-\frac{x^2-S}{2x} \\ 
   & = \frac{2x^2-x^2-S}{2x} \\
   & = \frac{x^2-S}{2x} \\
   & = \frac{1}{2}(x-S/x)
\end{aligned}
$$

The result is Heron's formula:
$$g(x) = \frac{1}{2}(x-S/x)$$

><mark>Heron's formula is a special case of Newton-Raphson's method.</mark>

[Back to top](#EEEN30101-Numerical-Analysis)

### Computational efficiency of Newton's method

As we did in [Heron's method - Convergence](#Heron's-method---Convergence), let us use Taylor's theorem to produce a polynomial approximation of Newton's method. This approximation, will allow us to explore the computational efficiency of the algorithm).

However, we need more accuracy this time, so we will be taking the first three terms of Taylor's expansion:

$$f(x + \delta x) = f(x) + \delta x f'(x) + \frac{\delta x ^2}{2}f''(\xi)$$

As before $\xi$ should be an interval between $x$ and $\delta x$.

We will also define our current estimation of a root $x_k$ as:

$$x_k=x$$

And the actual value of the root (i.e., the solution) $x^*$ as:

$$ x + \delta x = x^*  \therefore  \delta = x^*-x$$

As we should remember, the value of a function becomes zero at the root.

$$f(x^*) = 0$$

We use this information to update our Taylor's expansion:

$$
\begin{aligned}
  0 & = f(x_k) + (x^*-x_k)f'(x_k) + \frac{(x^*-x_k)^2}{2}f''(\xi) \\
   & =  \frac{f(x_k)}{f'(x_k)} + \frac{(x^*-x_k)f'(x_k)}{f'(x_k)} + \frac{(x^*-x_k)^2}{2f'(x_k)}f''(\xi)\\ 
   & =  x^*-x_k +\frac{f(x_k)}{f'(x_k)}+ \frac{(x^*-x_k)^2}{2}\frac{f''(\xi)}{f'(x_k)}
\end{aligned}
$$

As, based on Newton's method, $-x_{k+1}= -x_k +\frac{f(x_k)}{f'(x_k)}$, we get:
$$0 = x^*-x_{k+1} + \frac{(x^*-x_k)^2}{2}\frac{f''(\xi)}{f'(x_k)}$$

We can now rewrite the equation in terms of the error ($e_k = |x^*-x_{k+1}|$):

$$|x^*-x_{k+1}| = \frac{(x^*-x_k)^2}{2}\frac{|f''(\xi)|}{|f'(x_k)|}$$

$$e_{k+1} = \frac{e_k^2}{2}\frac{|f''(\xi)|}{|f'(x_k)|}$$

><mark>What this formula is telling us is that the error in Newton's method, decreases in a quadratic manner (i.e., it is a quadratic function due to $e^2_k$) after each iteration.</mark>

[Back to top](#EEEN30101-Numerical-Analysis)

## Conclusion

At the end of this week's lecture and after going through the notebook, you should be able to address the following questions:<br/>
- Heron's method: <br/>
      - What are Taylor's expansions used for?<br/>
      - What are the conditions that allow Heron's method to converge?<br/>
      - What are the conditions that allow Heron's method to converge faster?<br/><br/>
- Root finding algorithms and the Bisection algorithm:<br/>
      - What are roots?<br/>
      - What are root finding algorithms?<br/>
      - What is the Bisection algorithm?<br/><br/>
- Secant method:<br/>
      - What is the Secant method?<br/>
      - What are the conditions that improve the performance of the Secant method?<br/><br/>
- Newton-Raphson's method: <br/>
      - What is Newton's method?<br/>
      - How computationally efficient is Newton's method compared with Bisection, Secant and Heron's methods?

If you cannot answer these questions, you may want to check again this notebook and the lecture notes.

[Back to top](#EEEN30101-Numerical-Analysis)